In [1]:
from src.evaluation import foo

foo()

importing foo from module.


In [ ]:
# Kaggle API requires an access token
# https://github.com/Kaggle/kagglehub

# export KAGGLE_API_TOKEN=xxxxxxxxxxxxxx 

In [ ]:
from pathlib import Path
import os

project_root = Path.cwd()
data_dir = project_root / "data"
data_dir.mkdir(parents=True, exist_ok=True)

os.environ["KAGGLEHUB_CACHE"] = str(data_dir)

import kagglehub

dataset_path = kagglehub.dataset_download("zlatan599/mushroom1")

print(os.environ.get("KAGGLEHUB_CACHE_DIR"))



/workspace/data
